# Binary Classfication of Sonar return Project

### Importing modules

In [61]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [62]:
#initializing random seed
seed = 3
np.random.seed(seed)

### Importing data and creating a dataframe

In [63]:
!wget -c "https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data"

df = pd.read_csv('sonar.all-data',header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,0.1609,0.1582,0.2238,0.0645,0.0660,0.2273,0.3100,0.2999,0.5078,0.4797,0.5783,0.5071,0.4328,0.5550,0.6711,0.6415,0.7104,0.8080,0.6791,0.3857,0.1307,0.2604,0.5121,0.7547,0.8537,0.8507,0.6692,0.6097,0.4943,0.2744,0.0510,0.2834,0.2825,0.4256,0.2641,0.1386,0.1051,0.1343,0.0383,0.0324,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,0.4918,0.6552,0.6919,0.7797,0.7464,0.9444,1.0000,0.8874,0.8024,0.7818,0.5212,0.4052,0.3957,0.3914,0.3250,0.3200,0.3271,0.2767,0.4423,0.2028,0.3788,0.2947,0.1984,0.2341,0.1306,0.4182,0.3835,0.1057,0.1840,0.1970,0.1674,0.0583,0.1401,0.1628,0.0621,0.0203,0.0530,0.0742,0.0409,0.0061,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,0.6333,0.7060,0.5544,0.5320,0.6479,0.6931,0.6759,0.7551,0.8929,0.8619,0.7974,0.6737,0.4293,0.3648,0.5331,0.2413,0.5070,0.8533,0.6036,0.8514,0.8512,0.5045,0.1862,0.2709,0.4232,0.3043,0.6116,0.6756,0.5375,0.4719,0.4647,0.2587,0.2129,0.2222,0.2111,0.0176,0.1348,0.0744,0.0130,0.0106,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,0.0881,0.1992,0.0184,0.2261,0.1729,0.2131,0.0693,0.2281,0.4060,0.3973,0.2741,0.3690,0.5556,0.4846,0.3140,0.5334,0.5256,0.2520,0.2090,0.3559,0.6260,0.7340,0.6120,0.3497,0.3953,0.3012,0.5408,0.8814,0.9857,0.9167,0.6121,0.5006,0.3210,0.3202,0.4295,0.3654,0.2655,0.1576,0.0681,0.0294,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,0.4152,0.3952,0.4256,0.4135,0.4528,0.5326,0.7306,0.6193,0.2032,0.4636,0.4148,0.4292,0.5730,0.5399,0.3161,0.2285,0.6995,1.0000,0.7262,0.4724,0.5103,0.5459,0.2881,0.0981,0.1951,0.4181,0.4604,0.3217,0.2828,0.2430,0.1979,0.2444,0.1847,0.0841,0.0692,0.0528,0.0357,0.0085,0.0230,0.0046,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [64]:
# creating a dataset
dataset = df.values

# split into dependen/target and independent variables
X = dataset[:,:60].astype(float)
y = dataset[:,60]
print(X[0:5])
print(y[0:5])

[[0.02   0.0371 0.0428 0.0207 0.0954 0.0986 0.1539 0.1601 0.3109 0.2111
  0.1609 0.1582 0.2238 0.0645 0.066  0.2273 0.31   0.2999 0.5078 0.4797
  0.5783 0.5071 0.4328 0.555  0.6711 0.6415 0.7104 0.808  0.6791 0.3857
  0.1307 0.2604 0.5121 0.7547 0.8537 0.8507 0.6692 0.6097 0.4943 0.2744
  0.051  0.2834 0.2825 0.4256 0.2641 0.1386 0.1051 0.1343 0.0383 0.0324
  0.0232 0.0027 0.0065 0.0159 0.0072 0.0167 0.018  0.0084 0.009  0.0032]
 [0.0453 0.0523 0.0843 0.0689 0.1183 0.2583 0.2156 0.3481 0.3337 0.2872
  0.4918 0.6552 0.6919 0.7797 0.7464 0.9444 1.     0.8874 0.8024 0.7818
  0.5212 0.4052 0.3957 0.3914 0.325  0.32   0.3271 0.2767 0.4423 0.2028
  0.3788 0.2947 0.1984 0.2341 0.1306 0.4182 0.3835 0.1057 0.184  0.197
  0.1674 0.0583 0.1401 0.1628 0.0621 0.0203 0.053  0.0742 0.0409 0.0061
  0.0125 0.0084 0.0089 0.0048 0.0094 0.0191 0.014  0.0049 0.0052 0.0044]
 [0.0262 0.0582 0.1099 0.1083 0.0974 0.228  0.2431 0.3771 0.5598 0.6194
  0.6333 0.706  0.5544 0.532  0.6479 0.6931 0.6759 0.7551 0.892

tranforming y values to integer values using LabelEncoder

In [65]:
encoder = LabelEncoder()

encoder.fit(y)
encoded_y = encoder.transform(y)

### Creating base model

In [66]:
def base_model():
  #creating model
  model = Sequential()
  model.add(Dense(60,input_dim = 60 ,kernel_initializer='normal',activation='relu'))
  model.add(Dense(1,kernel_initializer='normal',activation='sigmoid'))
  #compile model
  model.compile(loss = 'binary_crossentropy',optimizer = 'adam', metrics = ['accuracy'])
  return model
  

In [67]:
#evaluate the model
estimator = KerasClassifier(build_fn = base_model,epochs = 100, batch_size = 5, verbose = 0)
kfold = StratifiedKFold(n_splits = 10,shuffle=True, random_state=seed)
results = cross_val_score(estimator,X,encoded_y,cv = kfold)
print(f"Baseline  model accuracy: {(results.mean()*100):.2f}% +/- ({(results.std()*100):.2f})")

Baseline  model accuracy: 82.26% +/- (8.25)


### Optimizing our base model

Let's first normalize our x data, for this we will use StandardScalar and pipeline and pass it to the cross_val_score func.

In [68]:
estimator1 = []
estimator1.append(('standardize',StandardScaler()))
estimator1.append(('mlp',KerasClassifier(build_fn=base_model,epochs = 100, batch_size = 5,verbose=0)))
pipeline = Pipeline(estimator1)
results1 = cross_val_score(pipeline,X,encoded_y,cv = kfold)
print(f"new model accuracy: {results1.mean()*100: .2f}% +/- ({results1.std()*100: .2f})")

new model accuracy:  87.07% +/- ( 7.65)


Let's try to optimize further
we will decrease the no. of neurons in the first hidden layer forcing it pick best features from the input.

In [69]:
# creating a smaller network
def small_model():
  #create model
  model = Sequential()
  model.add(Dense(30, input_dim = 60,kernel_initializer='normal',activation='relu'))
  model.add(Dense(1,kernel_initializer='normal',activation='sigmoid'))
  #compile model
  model.compile(loss = 'binary_crossentropy',optimizer='adam',metrics = ['accuracy'])
  return model

In [70]:
#running the new small model
estimator2 = []
estimator2.append(('standardize',StandardScaler()))
estimator2.append(('mlp',KerasClassifier(build_fn=small_model,epochs = 100, batch_size=5,verbose= 0)))
pipeline = Pipeline(estimator2)
kfold = StratifiedKFold(n_splits = 10,shuffle = True,random_state=seed)
results2 = cross_val_score(pipeline,X,encoded_y,cv = kfold)
print(f"new small model accuracy: {results2.mean()*100: .2f}% +/- ({results2.std()*100: .2f})")

new small model accuracy:  84.19% +/- ( 7.31)


let's optimize by increasing the size of the network

In [73]:
def large_network():
  #create model
  model = Sequential()
  model.add(Dense(60, input_dim = 60,kernel_initializer='normal',activation='relu'))
  model.add(Dense(30,kernel_initializer='normal',activation='relu'))
  model.add(Dense(1,kernel_initializer='normal',activation='sigmoid'))
  #compile model
  model.compile(loss = 'binary_crossentropy',optimizer='adam',metrics = ['accuracy'])
  return model

In [74]:
#running the new small model
estimator3 = []
estimator3.append(('standardize',StandardScaler()))
estimator3.append(('mlp',KerasClassifier(build_fn=large_network,epochs = 100, batch_size=5,verbose= 0)))
pipeline3 = Pipeline(estimator3)
kfold = StratifiedKFold(n_splits = 10,shuffle = True,random_state=seed)
results3 = cross_val_score(pipeline3,X,encoded_y,cv = kfold)
print(f"new small model accuracy: {results3.mean()*100: .2f}% +/- ({results3.std()*100: .2f})")

new small model accuracy:  87.07% +/- ( 7.65)


### Result

we achieved best mean accuracy of 87.07%

with standard deviation of 7.65

using a neural network containing one hidden layer of 60 neurons.